In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

results = {}
log_directory = "tapp-resultaten"
output_directory = "out"

shutil.rmtree(os.path.join(os.getcwd(), output_directory))
os.mkdir(os.path.join(os.getcwd(), output_directory))

# Plotting single graph using a boxplot
def create_graph(unit, experiment):    
    labels = list(experiment.keys())
    for index in range(len(labels)):
        labels[index] = labels[index][:-4]
    plt.boxplot(experiment.values(), vert=False, patch_artist=True, boxprops=dict(facecolor="lightgray"), showfliers=False)
    plt.title("Unit: " + unit)    
    plt.legend()
    plt.xlabel(unit)
    plt.yticks(range(1, len(labels) + 1), labels)
    
    fig = plt.gcf()
    plt.show()
    plt.draw()
    # fig.savefig(output_directory + "/" + unit + ".png")
    plt.close()

    for key, value in experiment.items():
        average = np.average(value)
        min = np.min(value)
        max = np.max(value)
        print(f"Experiment: {key[:-4]}, average: {average}, minimun: {min}, maximun: {max}")

# Plot many graphs with list and experiment name
def create_graphs(experiment_name, result_values):
    for unit, values in result_values.items():
        create_graph(experiment_name, unit, values)

# Retrieval logs from directory
for file in os.listdir(log_directory):
    filename = os.fsdecode(file)
    results[filename] = pd.read_csv(log_directory + "/" + file)
    results[filename]['mW'] = results[filename]['V'] * results[filename]['µA'] / 1e3

"""
{
    "V": {
        "DEEP_SLEEP": [3.2, 3.3, 3.4],
        "LOG___WFI": [3.2, 3.3, 3.4],
        "LOG_FULL_POWER": [3.2, 3.3, 3.4]
    },
    "µA": {
        "DEEP_SLEEP": [120, 130, 140],
        "LOG___WFI": [120, 130, 140],
        "LOG_FULL_POWER": [120, 130, 140]
    },
    "mW": {
        "DEEP_SLEEP": [1, 2, 3],
        "LOG___WFI": [1, 2, 3],
        "LOG_FULL_POWER": [1, 2, 3]
    }
}
"""

# Plotting graphs
pivoted_result = {}
for filename, data in results.items():
    for key, values in data.items():
        if key not in pivoted_result:
            pivoted_result[key] = {}
        pivoted_result[key][filename] = values

for unit, experiment in pivoted_result.items():
    create_graph(unit, experiment)

for key in pivoted_result:
    if "LOG_FULL_POWER.CSV" in pivoted_result[key]:
        pivoted_result[key].pop("LOG_FULL_POWER.CSV", None)
    if "LOG___WFI.CSV" in pivoted_result[key]:
        pivoted_result[key].pop("LOG___WFI.CSV", None)

for unit, experiment in pivoted_result.items():
    create_graph(unit, experiment)

